In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np



sns.set_style('whitegrid')

In [54]:
from scipy.special import rel_entr

def kl_divergence(solution: np.ndarray, submission: np.ndarray, epsilon: float=10**-15, micro_average: bool=True, sample_weights: pd.Series =None):
    
     # Prevent issue with populating int columns with floats
        
    solution= solution.astype(float)

     # clipping the min prevents users from playing games with the 20th decimal place of predictions.
    submission = np.clip(submission, epsilon, 1 )

    if micro_average:
        return np.average(rel_entr(solution, submission).sum(axis=1), weights=sample_weights) #this one is used by default
    else:
        return np.average(rel_entr(solution, submission).mean(axis=0))

In [34]:
pd.read_parquet('features_folder/unique_eegid/train_unique_eegid_mrgd_votes_feats_from_eeg_to_spec.parquet').iloc[:,:-7]

,eeg_id,offset_bins,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,...,eeg_std_f502_10s,eeg_std_f503_10s,eeg_std_f504_10s,eeg_std_f505_10s,eeg_std_f506_10s,eeg_std_f507_10s,eeg_std_f508_10s,eeg_std_f509_10s,eeg_std_f510_10s,eeg_std_f511_10s
0,568657,0,0,0.0,789577333,0,0.0,1825637311,20654,Other,...,0.183161,0.182791,0.179814,0.177479,0.177148,0.177129,0.177090,0.176519,0.177114,0.175883
1,642382,0,0,0.0,14960202,12,1008.0,3254468733,5955,Other,...,0.083412,0.085572,0.084796,0.079746,0.077148,0.077831,0.081943,0.086489,0.087046,0.085633
2,751790,0,0,0.0,618728447,4,908.0,2898467035,38549,GPD,...,0.133600,0.135571,0.137783,0.134362,0.134729,0.137103,0.142233,0.145995,0.145307,0.138285
3,778705,0,0,0.0,52296320,0,0.0,3255875127,40955,Other,...,0.097919,0.099461,0.100075,0.098297,0.095022,0.093828,0.094831,0.098256,0.094028,0.089804
4,1629671,0,0,0.0,2036345030,0,0.0,3715807879,37481,Seizure,...,0.086586,0.091101,0.096156,0.097291,0.096886,0.097578,0.099329,0.099535,0.096731,0.095313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15048,4293354003,0,0,0.0,1188113564,0,0.0,447244163,16610,GRDA,...,0.086978,0.095460,0.106431,0.105327,0.092422,0.088898,0.086935,0.084466,0.085176,0.084451
15049,4293843368,0,0,0.0,1549502620,0,0.0,1618953053,15065,GRDA,...,0.096788,0.098459,0.098498,0.098957,0.099933,0.098893,0.098645,0.100680,0.100817,0.097390
15050,4294455489,0,0,0.0,2105480289,0,0.0,469526364,56,Other,...,0.129369,0.129540,0.136376,0.132496,0.130420,0.130445,0.130406,0.137638,0.130468,0.129178
15051,4294858825,0,0,0.0,657299228,0,0.0,561576493,4312,Other,...,0.094562,0.096131,0.098471,0.097480,0.096054,0.095164,0.093635,0.093476,0.089711,0.088837


In [40]:
#the train dataframe with minimal filtering and no features
df=pd.read_csv('train_final_less_filtering.csv')

#dataframe with eeg-derived features, 
#setting index of the one saved in train.csv to the one in train_final_less_filtering.csv
#to allow fast dropping of the row in spec derived feature file
df_train=pd.read_parquet('features_folder/unique_eegid/train_unique_eegid_mrgd_votes_feats_from_eeg_to_spec.parquet')
df_train=df[['Unnamed: 0', 'label_id']].merge(df_train, how='inner', left_on='label_id', right_on='label_id')
df_train=df_train.set_index('Unnamed: 0')
df_train.index.name=None

#dataframe with kaggle spec derived features
df_train2=pd.read_parquet('train_features_from_kaggle_spec.parquet').loc[df_train.index]

#dataframe with spec from eeg  and eeg features
df_train3=pd.read_parquet('features_folder/features_from_eeg_spectrograms_without_kaggle_spec_data/train_mrgd_votes_no_ovlp_bfsplt_feats_from_eeg_to_spec.parquet')
df_train3=df[['Unnamed: 0', 'label_id']].merge(df_train3, how='inner', left_on='label_id', right_on='label_id')
df_train3=df_train3.set_index('Unnamed: 0')
df_train3.index.name=None
df_train3=df_train3[df_train3.columns[465:]]


#merging all features together
df_train=df_train.join(other=df_train2[df_train2.columns[3:]], how='left')
df_train

,label_id,eeg_id,offset_bins,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,patient_id,expert_consensus,...,RP_18.16_min_20s,RP_18.36_min_20s,RP_18.55_min_20s,RP_18.75_min_20s,RP_18.95_min_20s,RP_19.14_min_20s,RP_19.34_min_20s,RP_19.53_min_20s,RP_19.73_min_20s,RP_19.92_min_20s
0,127492639,1628180742,0,0,0.0,353733,0,0.0,42516,Seizure,...,0.514,0.492,0.645,0.481,0.490,0.504,0.415,0.482,0.458,0.445
11,557980729,722738444,0,0,0.0,999431,0,0.0,56885,LRDA,...,0.175,0.191,0.190,0.191,0.166,0.140,0.109,0.109,0.114,0.102
22,4099147263,387987538,0,0,0.0,1084844,0,0.0,4264,LRDA,...,0.187,0.138,0.106,0.100,0.093,0.101,0.121,0.123,0.157,0.149
28,1963161945,2175806584,0,0,0.0,1219001,0,0.0,23435,Seizure,...,3.335,3.019,2.847,2.759,2.432,2.725,2.643,2.678,2.853,2.760
30,3631726128,1626798710,0,0,0.0,1219001,2,74.0,23435,Seizure,...,2.269,1.806,1.725,1.515,1.403,1.401,1.386,1.532,1.661,2.014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106780,4272062867,3910994355,0,0,0.0,2146798838,0,0.0,28488,LPD,...,0.404,0.377,0.352,0.307,0.267,0.255,0.214,0.210,0.212,0.197
106781,2587113091,3938393892,0,0,0.0,2146798838,1,60.0,28488,LPD,...,0.376,0.351,0.314,0.271,0.244,0.246,0.257,0.287,0.303,0.298
106783,2394534310,1850739625,0,0,0.0,2146798838,3,162.0,28488,LPD,...,0.293,0.277,0.267,0.285,0.334,0.346,0.334,0.326,0.294,0.242
106784,1216355904,1306668185,0,0,0.0,2147312808,0,0.0,57480,LPD,...,0.643,0.655,0.690,0.694,0.737,0.859,0.845,0.843,0.746,0.609


In [36]:
#some rows have NaN values - these come from kaggle spec-derived features. Total 30
df.set_index('Unnamed: 0').loc[df_train[df_train.isna().sum(axis=1)>0].index]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,eeg_50sec_nan_row_count,eeg_10sec_nan_row_count,spectrogram_600sec_nan_row_count,spectrogram_10sec_nan_row_count,total_votes
1701,2536169515,0,0.0,19384736,55,1394.0,1305119316,56450,LPD,0,3,0,0,0,0,0,0,92,6,3
11571,4046938588,0,0.0,225347743,2,632.0,4062091881,29441,Other,0,0,0,0,0,1,0,0,150,6,1
11573,1225901968,0,0.0,225347743,4,824.0,2380909638,29441,LPD,0,3,0,0,0,0,0,0,150,6,3
24017,191408832,1,8.0,470580984,15,683.0,414130193,56450,LPD,0,3,0,0,0,0,0,0,124,6,3
24385,3350961119,0,0.0,480127814,17,717.0,713604191,56450,LPD,0,3,0,0,0,0,0,0,142,6,3
25074,1944688715,5,26.0,492373983,241,1394.0,2196981498,42180,Seizure,3,0,0,0,0,0,0,0,127,6,3
32874,3513444117,0,0.0,638972073,38,1746.0,29909846,38549,Other,0,0,2,0,2,9,0,0,33,6,13
40784,1410292431,3,24.0,784750288,4,283.0,1619190593,29441,LPD,0,4,0,0,0,0,0,0,25,6,4
45704,2132804048,0,0.0,885693423,42,1042.0,4250276239,37979,LRDA,1,0,0,8,0,5,0,0,151,6,14
48532,3166448814,0,0.0,942244950,26,2317.0,1533383301,17408,GPD,0,0,4,0,0,0,0,0,59,6,4


In [37]:
#features names. all features
feature_cols=list(df_train.columns[17:])
#features from kaggle spec
spec_features=df_train2.columns[3:]
#features from eeg
eeg_features=feature_cols[:448]
#features from specs from eegs
mels_features=df_train3.columns
#columns with probability values
vote_cats=['seizure_vote', 'lpd_vote',	'gpd_vote',	'lrda_vote',	'grda_vote',	'other_vote']
# encoding of the expert consensus into integers
codes = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}


In [38]:
select_features=['T5-O1.skewness',
 'F4-C4.samp_en',
 'C4-P4.samp_en',
 'T5-O1.rel_bp_delta',
 'C4-P4.kurtosis',
 'Fp1-F3.abs_bp_delta',
 'LP_10.16_mean_20s',
 'F3-C3.bp_delta_theta',
 'F3-C3.rel_bp_delta',
 'T5-O1.abs_bp_beta',
 'F4-C4.abs_bp_delta',
 'T6-O2.skewness',
 'RL_1.56_min_10m',
 'T5-O1.bp_delta_beta',
 'eeg_std_f302_10s',
 'P4-O2.samp_en',
 'F4-C4.abs_bp_theta',
 'RP_19.92_min_10m',
 'T4-T6.rel_bp_theta',
 'LP_3.52_min_10m',
 'C4-P4.rel_bp_delta',
 'T3-T5.rel_bp_delta']

In [49]:
#logistic regression on eeg + kaggle spec features
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler


n_splits=5
sgkf=StratifiedGroupKFold(n_splits=n_splits, random_state=216, shuffle=True)
gen=sgkf.split(X=df_train, y=df_train['expert_consensus'], 
                        groups=df_train['patient_id'])

features=list(spec_features)+list(eeg_features)

df_scores=pd.DataFrame(index=list(range(n_splits)), columns=['kl_div', 'accuracy'], data=None)


for i, (train_index, test_index) in enumerate(gen):

    n_classes = 6

    print(f'running fold {i}')
    X_train=df_train.iloc[train_index].dropna()[features]
    Y_train=df_train.iloc[train_index].dropna().expert_consensus.map(codes)
    Y_train_proba=df_train.iloc[train_index].dropna()[vote_cats]

    n_samples, n_classes=Y_train_proba.shape
    X_upsampled = np.array(X_train).repeat(n_classes, axis=0)
    Y_direct = np.tile(range(n_classes), n_samples)
    
    weight=np.array(df_train.iloc[train_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    sample_weight = np.array(Y_train_proba).ravel()*weight.repeat(6, axis=0)
    
    X_test=df_train.iloc[test_index].dropna()[features]
    Y_test_cat=df_train.iloc[test_index].dropna().expert_consensus.map(codes)
    Y_test_proba=df_train.iloc[test_index].dropna()[vote_cats]

    pipel=Pipeline([('scale', StandardScaler()), ('logreg',  LogisticRegression(penalty=None,max_iter=1000))])
    pipel.fit(X_upsampled,Y_direct, **{'logreg__sample_weight': sample_weight})
    pred=pipel.predict_proba(X_test.values)

    weight=np.array(df_train.iloc[test_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    df_scores.loc[i,'kl_div']=kl_divergence(Y_test_proba, pred)
    df_scores.loc[i,'accuracy']=pipel.score(X_test.values, Y_test_cat.values)

display(df_scores)
display(df_scores.mean())
#df_feature_imp=pd.DataFrame({'importance': pipel['logreg'].coef_[0], 'feature':features})
#display(df_feature_imp.sort_values(by='importance', ascending=False))

running fold 0


/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


running fold 1


/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


running fold 2


...

/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


running fold 4


/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,kl_div,accuracy
0,2.119092,0.483958
1,2.238553,0.434936
2,2.144694,0.4285
3,1.709389,0.50729
4,2.235587,0.436127


kl_div      2.089463
accuracy    0.458162
dtype: object

In [27]:
for i,c in enumerate(feature_cols):
    if 'vote' in c:
        print(i)

2489
2490
2491
2492
2493
2494
2495


In [49]:
#logistic regression on all features
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler


n_splits=5
sgkf=StratifiedGroupKFold(n_splits=n_splits, random_state=216, shuffle=True)
gen=sgkf.split(X=df_train, y=df_train['expert_consensus'], 
                        groups=df_train['patient_id'])
#features=select_features.feature_names.values
features=list(spec_features)+list(eeg_features)+list(mels_features)

df_scores=pd.DataFrame(index=list(range(n_splits)), columns=['kl_div', 'accuracy'], data=None)


for i, (train_index, test_index) in enumerate(gen):

    n_classes = 6

    print(f'running fold {i}')
    X_train=df_train.iloc[train_index].dropna()[features]
    Y_train=df_train.iloc[train_index].dropna().expert_consensus.map(codes)
    Y_train_proba=df_train.iloc[train_index].dropna()[vote_cats]

    n_samples, n_classes=Y_train_proba.shape
    X_upsampled = np.array(X_train).repeat(n_classes, axis=0)
    Y_direct = np.tile(range(n_classes), n_samples)
    
    weight=np.array(df_train.iloc[train_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    sample_weight = np.array(Y_train_proba).ravel()*weight.repeat(6, axis=0)
    
    X_test=df_train.iloc[test_index].dropna()[features]
    Y_test_cat=df_train.iloc[test_index].dropna().expert_consensus.map(codes)
    Y_test_proba=df_train.iloc[test_index].dropna()[vote_cats]

    pipel=Pipeline([('scale', StandardScaler()), ('logreg',  LogisticRegression(max_iter=1000))])
    pipel.fit(X_upsampled,Y_direct, **{'logreg__sample_weight': sample_weight})
    pred=pipel.predict_proba(X_test.values)

    weight=np.array(df_train.iloc[test_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    df_scores.loc[i,'kl_div']=kl_divergence(Y_test_proba, pred)
    df_scores.loc[i,'accuracy']=pipel.score(X_test.values, Y_test_cat.values)

display(df_scores)
display(df_scores.mean())
#df_feature_imp=pd.DataFrame({'importance': pipel['logreg'].coef_[0], 'feature':features})
#display(df_feature_imp.sort_values(by='importance', ascending=False))

running fold 0


/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


running fold 1


/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


running fold 2


...

/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


running fold 4


/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,kl_div,accuracy
0,1.60785,0.460186
1,1.69763,0.460109
2,1.607578,0.510943
3,1.45838,0.492879
4,1.557119,0.470178


kl_div      1.585711
accuracy    0.478859
dtype: object

In [22]:
pred2=pipel.predict_proba(pd.read_parquet('all_features.parquet').dropna()[feature_cols])

kl_divergence(pd.read_parquet('all_features.parquet').dropna()[vote_cats].values, pred2)

/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


0.0926014572805251

In [33]:
#pd.DataFrame(pipel['logreg'].intercept_, columns=['intercept']).to_csv('weights_intercept')

In [19]:
# import pickle
# with open('logistic_regression_all_features_uniq_eeg_l2.pickle', 'wb') as f:
#      pickle.dump(pipel['logreg'], f, pickle.HIGHEST_PROTOCOL)


In [22]:
#logistic regression on kaggle spec + mels spec features
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler


n_splits=5
sgkf=StratifiedGroupKFold(n_splits=n_splits, random_state=216, shuffle=True)
gen=sgkf.split(X=df_train, y=df_train['expert_consensus'], 
                        groups=df_train['patient_id'])
#features=select_features.feature_names.values
features=list(mels_features)+list(spec_features)

df_scores=pd.DataFrame(index=list(range(n_splits)), columns=['kl_div', 'accuracy'], data=None)


for i, (train_index, test_index) in enumerate(gen):

    n_classes = 6

    print(f'running fold {i}')
    X_train=df_train.iloc[train_index].dropna()[features]
    Y_train=df_train.iloc[train_index].dropna().expert_consensus.map(codes)
    Y_train_proba=df_train.iloc[train_index].dropna()[vote_cats]

    n_samples, n_classes=Y_train_proba.shape
    X_upsampled = np.array(X_train).repeat(n_classes, axis=0)
    Y_direct = np.tile(range(n_classes), n_samples)
    
    weight=np.array(df_train.iloc[train_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    sample_weight = np.array(Y_train_proba).ravel()*weight.repeat(6, axis=0)
    
    X_test=df_train.iloc[test_index].dropna()[features]
    Y_test_cat=df_train.iloc[test_index].dropna().expert_consensus.map(codes)
    Y_test_proba=df_train.iloc[test_index].dropna()[vote_cats]

    pipel=Pipeline([('scale', StandardScaler()), ('logreg',  LogisticRegression(penalty=None,max_iter=1000))])
    pipel.fit(X_upsampled,Y_direct, **{'logreg__sample_weight': sample_weight})
    pred=pipel.predict_proba(X_test.values)

    weight=np.array(df_train.iloc[test_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    df_scores.loc[i,'kl_div']=kl_divergence(Y_test_proba, pred)
    df_scores.loc[i,'accuracy']=pipel.score(X_test.values, Y_test_cat.values)

display(df_scores)
display(df_scores.mean())
df_feature_imp=pd.DataFrame({'importance': pipel['logreg'].coef_[0], 'feature':features})
#display(df_feature_imp.sort_values(by='importance', ascending=False))

running fold 0


/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


running fold 1


/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


running fold 2


...

/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


running fold 4


/Users/elagoda/opt/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,kl_div,accuracy
0,1.708071,0.429662
1,1.809195,0.410929
2,1.758521,0.477119
3,1.519311,0.4314
4,1.689811,0.425337


kl_div      1.696982
accuracy    0.434889
dtype: object

In [20]:
#logistic regression on select features
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler


n_splits=5
sgkf=StratifiedGroupKFold(n_splits=n_splits, random_state=216, shuffle=True)
gen=sgkf.split(X=df_train, y=df_train['expert_consensus'], 
                        groups=df_train['patient_id'])
#features=select_features.feature_names.values
features=select_features

df_scores=pd.DataFrame(index=list(range(n_splits)), columns=['kl_div', 'accuracy'], data=None)


for i, (train_index, test_index) in enumerate(gen):

    n_classes = 6

    print(f'running fold {i}')
    X_train=df_train.iloc[train_index].dropna()[features]
    Y_train=df_train.iloc[train_index].dropna().expert_consensus.map(codes)
    Y_train_proba=df_train.iloc[train_index].dropna()[vote_cats]

    n_samples, n_classes=Y_train_proba.shape
    X_upsampled = np.array(X_train).repeat(n_classes, axis=0)
    Y_direct = np.tile(range(n_classes), n_samples)
    
    weight=np.array(df_train.iloc[train_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    sample_weight = np.array(Y_train_proba).ravel()*weight.repeat(6, axis=0)
    
    X_test=df_train.iloc[test_index].dropna()[features]
    Y_test_cat=df_train.iloc[test_index].dropna().expert_consensus.map(codes)
    Y_test_proba=df_train.iloc[test_index].dropna()[vote_cats]

    pipel=Pipeline([('scale', StandardScaler()), ('logreg',  LogisticRegression(penalty=None,max_iter=1000))])
    pipel.fit(X_upsampled,Y_direct, **{'logreg__sample_weight': sample_weight})
    pred=pipel.predict_proba(X_test.values)

    weight=np.array(df_train.iloc[test_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    df_scores.loc[i,'kl_div']=kl_divergence(Y_test_proba, pred)
    df_scores.loc[i,'accuracy']=pipel.score(X_test.values, Y_test_cat.values)

display(df_scores)
display(df_scores.mean())
df_feature_imp=pd.DataFrame({'importance': pipel['logreg'].coef_[0], 'feature':features})
#display(df_feature_imp.sort_values(by='importance', ascending=False))

running fold 0
running fold 1
running fold 2
running fold 3
running fold 4


,kl_div,accuracy
0,1.079739,0.469144
1,1.189693,0.419672
2,1.073131,0.561082
3,1.133165,0.476207
4,1.091763,0.464084


kl_div      1.113498
accuracy    0.478038
dtype: object

In [18]:
#naive bayes eeg+kaggle spec features
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler

n_splits=5
sgkf=StratifiedGroupKFold(n_splits=n_splits, random_state=216, shuffle=True)
gen=sgkf.split(X=df_train, y=df_train['expert_consensus'], 
                        groups=df_train['patient_id'])
features=list(spec_features)+list(eeg_features)+list(mels_features)
df_scores=pd.DataFrame(index=list(range(n_splits)), columns=['kl_div', 'accuracy'], data=None)


for i, (train_index, test_index) in enumerate(gen):
    n_classes=6
    print(f'running fold {i}')
    X_train=df_train.iloc[train_index].dropna()[features]
    Y_train=df_train.iloc[train_index].dropna().expert_consensus.map(codes)
    Y_train_proba=df_train.iloc[train_index].dropna()[vote_cats]

    n_samples, n_classes=Y_train_proba.shape
    X_upsampled = np.array(X_train).repeat(n_classes, axis=0)
    Y_direct = np.tile(range(n_classes), n_samples)
    
    weight=np.array(df_train.iloc[train_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    sample_weight = np.array(Y_train_proba).ravel()*weight.repeat(6, axis=0)
    
    X_test=df_train.iloc[test_index].dropna()[features]
    Y_test_cat=df_train.iloc[test_index].dropna().expert_consensus.map(codes)
    Y_test_proba=df_train.iloc[test_index].dropna()[vote_cats]

    pipe=Pipeline([ ('naive',  GaussianNB())])
    pipe.fit(X_upsampled,Y_direct, **{'naive__sample_weight': sample_weight})
    pred=pipe.predict_proba(X_test.values)
    df_scores.loc[i,'kl_div']=kl_divergence(Y_test_proba, pred)
    df_scores.loc[i,'accuracy']=pipe.score(X_test.values, Y_test_cat.values)

display(df_scores)
display(df_scores.mean())
#df_feature_imp=pd.DataFrame({'importance': pipe['naive'].coef_[0], 'feature':features})
#display(df_feature_imp.sort_values(by='importance', ascending=False))

running fold 0
running fold 1
running fold 2
running fold 3
running fold 4


,kl_div,accuracy
0,26.791648,0.169874
1,30.154743,0.112933
2,31.457914,0.081178
3,29.49557,0.134074
4,29.172781,0.147148


kl_div      29.414531
accuracy     0.129041
dtype: object

In [17]:
#naive bayes on kaggle spec+mels features
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler

n_splits=5
sgkf=StratifiedGroupKFold(n_splits=n_splits, random_state=216, shuffle=True)
gen=sgkf.split(X=df_train, y=df_train['expert_consensus'], 
                        groups=df_train['patient_id'])
features=list(mels_features)+list(spec_features)
df_scores=pd.DataFrame(index=list(range(n_splits)), columns=['kl_div', 'accuracy'], data=None)


for i, (train_index, test_index) in enumerate(gen):
    n_classes=6
    print(f'running fold {i}')
    X_train=df_train.iloc[train_index].dropna()[features]
    Y_train=df_train.iloc[train_index].dropna().expert_consensus.map(codes)
    Y_train_proba=df_train.iloc[train_index].dropna()[vote_cats]

    n_samples, n_classes=Y_train_proba.shape
    X_upsampled = np.array(X_train).repeat(n_classes, axis=0)
    Y_direct = np.tile(range(n_classes), n_samples)
    
    weight=np.array(df_train.iloc[train_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    sample_weight = np.array(Y_train_proba).ravel()*weight.repeat(6, axis=0)
    
    X_test=df_train.iloc[test_index].dropna()[features]
    Y_test_cat=df_train.iloc[test_index].dropna().expert_consensus.map(codes)
    Y_test_proba=df_train.iloc[test_index].dropna()[vote_cats]

    pipe=Pipeline([ ('naive',  GaussianNB())])
    pipe.fit(X_upsampled,Y_direct, **{'naive__sample_weight': sample_weight})
    pred=pipe.predict_proba(X_test.values)
    df_scores.loc[i,'kl_div']=kl_divergence(Y_test_proba, pred)
    df_scores.loc[i,'accuracy']=pipe.score(X_test.values, Y_test_cat.values)

display(df_scores)
display(df_scores.mean())

running fold 0
running fold 1
running fold 2
running fold 3
running fold 4


,kl_div,accuracy
0,26.8951,0.06138
1,30.173237,0.113661
2,31.506849,0.079188
3,29.531984,0.133727
4,29.20151,0.145407


kl_div      29.461736
accuracy     0.106673
dtype: object

In [16]:
#naive bayes select features 
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler

n_splits=5
sgkf=StratifiedGroupKFold(n_splits=n_splits, random_state=216, shuffle=True)
gen=sgkf.split(X=df_train, y=df_train['expert_consensus'], 
                        groups=df_train['patient_id'])
features=select_features
df_scores=pd.DataFrame(index=list(range(n_splits)), columns=['kl_div', 'accuracy'], data=None)


for i, (train_index, test_index) in enumerate(gen):
    n_classes=6
    print(f'running fold {i}')
    X_train=df_train.iloc[train_index].dropna()[features]
    Y_train=df_train.iloc[train_index].dropna().expert_consensus.map(codes)
    Y_train_proba=df_train.iloc[train_index].dropna()[vote_cats]

    n_samples, n_classes=Y_train_proba.shape
    X_upsampled = np.array(X_train).repeat(n_classes, axis=0)
    Y_direct = np.tile(range(n_classes), n_samples)
    
    weight=np.array(df_train.iloc[train_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))
    sample_weight = np.array(Y_train_proba).ravel()*weight.repeat(6, axis=0)
    
    X_test=df_train.iloc[test_index].dropna()[features]
    Y_test_cat=df_train.iloc[test_index].dropna().expert_consensus.map(codes)
    Y_test_proba=df_train.iloc[test_index].dropna()[vote_cats]

    pipe=Pipeline([ ('naive',  GaussianNB())])
    pipe.fit(X_upsampled,Y_direct, **{'naive__sample_weight': sample_weight})
    pred=pipe.predict_proba(X_test.values)
    df_scores.loc[i,'kl_div']=kl_divergence(Y_test_proba, pred)
    df_scores.loc[i,'accuracy']=pipe.score(X_test.values, Y_test_cat.values)

display(df_scores)
display(df_scores.mean())

running fold 0
running fold 1
running fold 2
running fold 3
running fold 4


,kl_div,accuracy
0,12.308538,0.173855
1,9.885661,0.150455
2,13.549103,0.1154
3,12.663201,0.134074
4,15.104657,0.150196


kl_div      12.702232
accuracy     0.144796
dtype: object

In [15]:
#naive bayes on select features trained on expert consensus
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler

n_splits=5
sgkf=StratifiedGroupKFold(n_splits=n_splits, random_state=216, shuffle=True)
gen=sgkf.split(X=df_train, y=df_train['expert_consensus'], 
                        groups=df_train['patient_id'])
features=select_features
df_scores=pd.DataFrame(index=list(range(n_splits)), columns=['kl_div', 'accuracy'], data=None)


for i, (train_index, test_index) in enumerate(gen):
    print(f'running fold {i}')
    X_train=df_train.iloc[train_index].dropna()[features]
    Y_train=df_train.iloc[train_index].dropna().expert_consensus.map(codes)
    X_test=df_train.iloc[test_index].dropna()[features]
    Y_test_cat=df_train.iloc[test_index].dropna().expert_consensus.map(codes)
    Y_test_proba=df_train.iloc[test_index].dropna()[vote_cats]

    weight=np.array(df_train.iloc[train_index].dropna().total_votes.apply(lambda x: min(x/3, 1.0)))

    pipe=Pipeline([ ('naive',  GaussianNB())])
    pipe.fit(X_train,Y_train, **{'naive__sample_weight': weight})
    pred=pipe.predict_proba(X_test)
    df_scores.loc[i,'kl_div']=kl_divergence(Y_test_proba, pred, )
    df_scores.loc[i,'accuracy']=pipe.score(X_test, Y_test_cat)

display(df_scores)
display(df_scores.mean())
#df_feature_imp=pd.DataFrame({'importance': pipe['naive'].coef_[0], 'feature':features})
#display(df_feature_imp.sort_values(by='importance', ascending=False))

running fold 0
running fold 1
running fold 2
running fold 3
running fold 4


,kl_div,accuracy
0,17.594287,0.149303
1,15.880912,0.13224
2,20.465371,0.091524
3,19.590407,0.08371
4,17.889829,0.103178


kl_div      18.284161
accuracy     0.111991
dtype: object

In [55]:
#baseline model: predicting probability 1/6 for each category
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.model_selection import StratifiedGroupKFold


n_splits=5
sgkf=StratifiedGroupKFold(n_splits=n_splits, random_state=216, shuffle=True)
gen=sgkf.split(X=df_train, y=df_train['expert_consensus'], 
                        groups=df_train['patient_id'])

df_scores=pd.DataFrame(index=list(range(n_splits)), columns=['kl_div', 'accuracy'], data=None)


for i, (train_index, test_index) in enumerate(gen):
    print(f'running fold {i}')

    Y_test_proba=df_train.iloc[test_index][vote_cats]
    pred=np.full(Y_test_proba.shape, 1/6, dtype=float)
    df_scores.loc[i,'kl_div']=kl_divergence(Y_test_proba, pred)

display(df_scores)
display(df_scores.mean())

running fold 0
running fold 1
running fold 2
running fold 3
running fold 4


,kl_div,accuracy
0,1.415272,NaN
1,1.481652,NaN
2,1.502297,NaN
3,1.490395,NaN
4,1.469953,NaN


kl_div      1.471914
accuracy         NaN
dtype: object